In [62]:
from numpy import * 

### 训练算法：使用梯度上升找到最佳参数

In [63]:
#读取textSet.txt并处理
def loadDataSet():
    dataMat = []; labelMat = []
    fr = open('testSet.txt')
    for line in fr.readlines():
        #strip移除字符串头尾指定的字符（默认为空格），返回移除字符串头尾指定的字符生成的新字符串。
        # split()通过指定分隔符对字符串进行切片，返回分割后的字符串列表。
        lineArr = line.strip().split()
        #向数组添加列表，每个列表包含1.0和两个特征值
        dataMat.append([1.0, float(lineArr[0]), float(lineArr[1])])
        labelMat.append(int(lineArr[2]))
    return dataMat,labelMat
    

In [64]:
#sigmoid函数
def sigmoid(inX):
    return 1.0/(1+exp(-inX))


In [65]:
#梯度上升算法
def gradAscent(dataMatIn, classLabels):
    dataMatrix = mat(dataMatIn)             #numpy样本 数组（每个样本包含2个特征和第0维特征，共100个样本） convert to NumPy matrix
    labelMat = mat(classLabels).transpose() #类别标签，它是一个1×100的行向量，convert to NumPy matrix，并转置
    m,n = shape(dataMatrix)
    #向目标移动的步长
    alpha = 0.001
    #迭代次数
    maxCycles = 500
    #创建n×1的全1向量
    weights = ones((n,1))
    #循环迭代完成之后，返回训练好的回归系数
    for k in range(maxCycles):              #heavy on matrix operations
        h = sigmoid(dataMatrix*weights)     #矩阵乘法的结果作为sigmoid函数的参数
        error = (labelMat - h)              #vector subtraction 计算真实类别与预测类别的差值
        #更新回归系数
        weights = weights + alpha * dataMatrix.transpose()* error #matrix mult
    return weights

### 分析数据：画出决策边界 

In [66]:
#画出数据集和Logistic回归最佳拟合直线的函数
def plotBestFit(weights):
    import matplotlib.pyplot as plt
    #获取数据
    dataMat,labelMat=loadDataSet()
    #转换成数组
    dataArr = array(dataMat)
    #获得样本容量
    n = shape(dataArr)[0] 
    xcord1 = []; ycord1 = []
    xcord2 = []; ycord2 = []
    #循环遍历
    for i in range(n):
        #若类别标签为1，将其对应样本的第1维和第2维特征值分 别添加进xcord1和ycord1列表
        if int(labelMat[i])== 1:
            xcord1.append(dataArr[i,1]); ycord1.append(dataArr[i,2])
        #类别标签不为1，同上
        else:
            xcord2.append(dataArr[i,1]); ycord2.append(dataArr[i,2])
    #创建画布
    fig = plt.figure()
    #创建子图
    ax = fig.add_subplot(111)
    #散列图
    ax.scatter(xcord1, ycord1, s=30, c='red', marker='s')
    ax.scatter(xcord2, ycord2, s=30, c='green')
    #生成数组,源代码没有reshape（1，60），运行会报错，提示x和y维度不同，所以进行了修改，添加了reshape
    x = arange(-3.0, 3.0, 0.1).reshape(1,60)
    #sigmoid函数为0.5（0 = z = w0x0+w1x1+w2x2）时，是两个分类（类别1和类别0 ) 的分界处,在loadDataSet函数中，x0也即第0维特征被设置为了1，
    #，x1就是上一行的x，y = x2 = （-w0-w1x1）/w2 = （-w0-w1x）/w2 ，即下式。
    y = (-weights[0]-weights[1]*x)/weights[2]
    #画线
    ax.plot(x, y)
    #横轴和纵轴标签
    plt.xlabel('X1'); plt.ylabel('X2');
    plt.show()

### 训练算法改进：随机梯度上升 

梯度上升算法在每次更新回归系数时都需要遍历整个数据集，该方法在处理数据量较大时计算复杂度就太高了，
一种改进方法是一次仅用一个样本点来更新回归系数，该方法称为随机梯度上升算法


In [67]:
#随机梯度上升法版本1
#随机梯度上升算法与梯度上升算法在代码上很相似，但也有一些区别：第一，后
#者的变量h和误差error都是向量，而前者则全是数值；第 二 ，前者没有矩阵的转换过程，所有
#变量的数据类型都是N UmPy数组。
def stocGradAscent0(dataMatrix, classLabels):
    m,n = shape(dataMatrix)
    alpha = 0.01
    weights = ones(n)   #initialize to all ones
    for i in range(m):
        #print dataMatrix[i]
        #因为是数组，数组乘法和矩阵不一样，所以要用到sum函数
        h = sigmoid(sum(dataMatrix[i]*weights))
       # print h
        error = classLabels[i] - h
       # print error
        #更新回归系数
        weights = weights + alpha * error * dataMatrix[i]
        print (alpha * error * dataMatrix[i])
    return weights

In [68]:
dataArr,labels = loadDataSet()

In [69]:
#随机梯度上升法版本2
def stocGradAscent1(dataMatrix, classLabels, numIter=150):
    m,n = shape(dataMatrix)
    weights = ones(n)   #initialize to all ones
    #循环遍历，进算法还增加了一个迭代次数作为第3个参数。如果该参数没有给定的话，算法将
    #默认迭代150次。如果给定，那么算法将按照新的参数值进行迭代。
    for j in range(numIter):
        dataIndex = range(m)
        for i in range(m):
            #alpha在每次迭代的时候都会调整，缓解数据波动或者高频波动
            #虽然alpha会随着迭代次数不断减小，但永远不会减小到0，这是因为下式中
            #还存在一个常数项^ 必须这样做的原因是为了保证在多次迭代之后新数据仍然具有一定的影响
            alpha = 4/(1.0+j+i)+0.0001    #apha decreases with iteration, does not 
            #通过随机选取样本来更新回归系数。这种方法将减少周期性的波动
            randIndex = int(random.uniform(0,len(dataIndex)))#go to 0 because of the constant
            h = sigmoid(sum(dataMatrix[randIndex]*weights))
            error = classLabels[randIndex] - h
            weights = weights + alpha * error * dataMatrix[randIndex]
            #随机从列表中选出一个值，然后从列表中删掉该值（再进行下一次迭代)
            del(dataIndex[randIndex])
    return weights


In [70]:
#dataArr,labels = loadDataSet()

In [71]:
#wei = stocGradAscent0(dataArr,labels)

### 项目示例：从疝气病症预测病马的死亡率

除了部分指标主观和难以测量外，该数据还存在一个问题，数据集中有30%的值是缺失的

处理被据中的缺失值:

□使用可用特征的均值来填补缺失值；

□使用特殊值来±真补缺失值，如-1;

□ 忽略有缺失值的样本；

□使用相似样本的均值添补缺失值；

□使用另外的机器学习算法预测缺失值。
本例中的处理方法：1、对于缺失的特征值选择实数0来替换 2、一条数据的类别标签已经缺失，那么我们的简单做法是将该条数据丢弃。
原始的数据集经过预处理之后保存成两个文件：horseColicTest. txt和 horseColicTraining.txt


开发流程

收集数据: 给定数据文件 

准备数据: 用 Python 解析文本文件并填充缺失值

分析数据: 可视化并观察数据

训练算法: 使用优化算法，找到最佳的系数

测试算法: 为了量化回归的效果，需要观察错误率。根据错误率决定是否回退到训练阶段，
         通过改变迭代的次数和步长的参数来得到更好的回归系数
         
使用算法: 实现一个简单的命令行程序来手机马的症状并输出预测结果并非难事，
         这可以作为留给大家的一道习题

In [72]:
#回归分类函数 以回归系数和特征向量作为输入来计算对应的sigmoid值。如果sigmoid值大于0.5函数返回1，否则返回0。
def classifyVector(inX, weights):
    prob = sigmoid(sum(inX*weights))
    if prob > 0.5: return 1.0
    else: return 0.0

In [73]:
#函数colicTest()，是用于打开测试集和训练集，并对数据进行格式化处理的函数
#数据的最后一列仍然是类别标签。数据最初有三个类别标签，分别代表马的三种情况：“仍存活”、“已经死亡”和 “已经安乐死”。这里为了方便，
#将 “已经死亡”和 “已经安乐死”合 并 成 “未能存活”这 个 标 签
def colicTest():
    #打开训练及和测试集
    frTrain = open('horseColicTraining.txt'); frTest = open('horseColicTest.txt')
    #创建列表
    trainingSet = []; trainingLabels = []
    #循环遍历
    for line in frTrain.readlines():
        #去除空白符并以‘\t’进行分割
        currLine = line.strip().split('\t')
        lineArr =[]
        #将每一个样本的前21个数值一次添加进lineArr列表
        for i in range(21):
            lineArr.append(float(currLine[i]))
        trainingSet.append(lineArr)
        #将下表为21的数值（类别标签）添加进trainingLabels
        trainingLabels.append(float(currLine[21]))
    #利用随机梯度上升法进行训练，返回训练后得到的回归系数
    trainWeights = stocGradAscent1(array(trainingSet), trainingLabels, 1000)
    errorCount = 0; numTestVec = 0.0
    #循环遍历测试数据集
    for line in frTest.readlines():
        numTestVec += 1.0
        currLine = line.strip().split('\t')
        lineArr =[]
        for i in range(21):
            lineArr.append(float(currLine[i]))
        #进行分类，并对分类结果进行检查
        if int(classifyVector(array(lineArr), trainWeights))!= int(currLine[21]):
            errorCount += 1
    #计算错误率
    errorRate = (float(errorCount)/numTestVec)
    print "the error rate of this test is: %f" % errorRate
    #返回错误率
    return errorRate

In [74]:
#multiTeSt ()，其功能是调用函数 colicTest()10次并求结果的平均值
def multiTest():
    numTests = 10; errorSum=0.0
    for k in range(numTests):
        errorSum += colicTest()
    print "after %d iterations the average error rate is: %f" % (numTests, errorSum/float(numTests))
        

In [75]:
multiTest()

c:\python27\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


the error rate of this test is: 0.328358
the error rate of this test is: 0.298507
the error rate of this test is: 0.343284
the error rate of this test is: 0.328358
the error rate of this test is: 0.402985
the error rate of this test is: 0.343284
the error rate of this test is: 0.417910
the error rate of this test is: 0.283582
the error rate of this test is: 0.432836
the error rate of this test is: 0.238806
after 10 iterations the average error rate is: 0.341791
